<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/observability/LlamaDebugHandler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama Debug Handler

Here we showcase the capabilities of our LlamaDebugHandler in logging events as we run queries
within LlamaIndex.

**NOTE**: This is a beta feature. The usage within different classes and the API interface
    for the CallbackManager and LlamaDebugHandler may change!

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)

## Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader("./data/paul_graham/").load_data()

## Callback Manager Setup

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

## Trigger the callback with a query

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs, callback_manager=callback_manager
)
query_engine = index.as_query_engine()

**********
Trace: index_construction
    |_node_parsing ->  0.134458 seconds
      |_chunking ->  0.132142 seconds
    |_embedding ->  0.329045 seconds
    |_embedding ->  0.357797 seconds
**********


In [ ]:
response = query_engine.query("What did the author do growing up?")

**********
Trace: query
    |_query ->  2.198197 seconds
      |_retrieve ->  0.122185 seconds
        |_embedding ->  0.117082 seconds
      |_synthesize ->  2.075836 seconds
        |_llm ->  2.069724 seconds
**********


## Explore the Debug Information

The callback manager will log several start and end events for the following types:
- CBEventType.LLM
- CBEventType.EMBEDDING
- CBEventType.CHUNKING
- CBEventType.NODE_PARSING
- CBEventType.RETRIEVE
- CBEventType.SYNTHESIZE 
- CBEventType.TREE
- CBEventType.QUERY

The LlamaDebugHandler provides a few basic methods for exploring information about these events

In [ ]:
# Print info on the LLM calls during the summary index query
print(llama_debug.get_event_time_info(CBEventType.LLM))

EventStats(total_secs=2.069724, average_secs=2.069724, total_count=1)


In [ ]:
# Print info on llm inputs/outputs - returns start/end events for each LLM call
event_pairs = llama_debug.get_llm_inputs_outputs()
print(event_pairs[0][0])
print(event_pairs[0][1].payload.keys())
print(event_pairs[0][1].payload["response"])

CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.", additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Context information is below.\n---------------------\nWhat I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming.I didn\'t write essays.I wrote what beginning writers were supposed to write then, and probably still are: short stories.My stories were awful.They had hardly any plot, just characters with strong feelings, which I imagined mad

In [ ]:
# Get info on any event type
event_pairs = llama_debug.get_event_pairs(CBEventType.CHUNKING)
print(event_pairs[0][0].payload.keys())  # get first chunking start event
print(event_pairs[0][1].payload.keys())  # get first chunking end event

dict_keys([<EventPayload.CHUNKS: 'chunks'>])
dict_keys([<EventPayload.CHUNKS: 'chunks'>])


In [ ]:
# Clear the currently cached events
llama_debug.flush_event_logs()

## See Traces & Events for Agents

In [ ]:
# First create a tool for the agent
from llama_index.core.tools import QueryEngineTool

tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="PaulGrahamQuestionAnswer",
    description="Given a question about Paul Graham, will return an answer.",
)

In [ ]:
# Now construct the agent
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(
    tools=[tool], llm=llm, callback_manager=callback_manager
)

In [ ]:
response = agent.chat("What did Paul do growing up?")

**********
Trace: chat
    |_llm ->  1.169013 seconds
    |_query ->  2.357469 seconds
      |_retrieve ->  0.107983 seconds
        |_embedding ->  0.099368 seconds
      |_synthesize ->  2.24932 seconds
        |_llm ->  2.239481 seconds
    |_llm ->  2.153333 seconds
**********


In [ ]:
# works the same for async
response = await agent.achat("What did Paul do growing up?")

**********
Trace: chat
    |_llm ->  1.318663 seconds
    |_query ->  2.803533 seconds
      |_retrieve ->  0.121228 seconds
        |_embedding ->  0.116355 seconds
      |_synthesize ->  2.68217 seconds
        |_llm ->  2.676306 seconds
    |_llm ->  2.716374 seconds
**********


In [ ]:
# Clear the currently cached events
llama_debug.flush_event_logs()